#### Importing Necessary Libs

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

### Create a class MP to store the column information

In [3]:
class MP:

    def __init__(self, postal_code, borough, neighborhood):
        self.postal_code = postal_code
        self.borough = borough
        self.neighborhood = neighborhood

    def print(self):
        print(self.postal_code + " " + self.borough + " " + self.neighborhood)

#### Requesting Html page and initializing variables

In [4]:
url_doc = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
data = BeautifulSoup(url_doc.content, 'html.parser')
list_of_MP = []

#### Creating a list of extracted data 

In [5]:
for rows in data.table.findAll('tr'):
    dt = []
    index = 1
    for col in rows.findAll('td'):
        if col.get_text().strip() == "Not assigned" and index == 3:
            dt.append(dt[index-2])
        if col.get_text().strip() == "Not assigned":
            continue
        else:
            dt.append(col.get_text().strip())
        index += 1

    if len(dt) == 3:
        mp = MP(dt[0], dt[1], dt[2])
        list_of_MP.append(mp)
    dt.clear()

#### Removing Redundancy

In [6]:
for i in range(0, len(list_of_MP)-1):
    j = i + 1
    while j < len(list_of_MP):
        try:
            if list_of_MP[i].postal_code == list_of_MP[j].postal_code:
                list_of_MP[i].neighborhood = list_of_MP[j].neighborhood + ", " + list_of_MP[i].neighborhood
                list_of_MP.pop(j)
            else:
                j += 1
        except IndexError:
            print(IndexError)

### Creating DataFrame

In [8]:
df = pd.DataFrame([(i.postal_code, i.borough, i.neighborhood) for i in list_of_MP], 
                  columns=['PostalCode', 'Borough', 'Neighborhood'])
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Queen's Park


In [9]:
df.shape

(103, 3)